In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib
import os

# Paths
h5_path = r"C:\Users\sagni\Downloads\Disaster Prediction\disaster_data.h5"
model_save_path = r"C:\Users\sagni\Downloads\Disaster Prediction\disaster_predictor.pkl"
prediction_map_path = r"C:\Users\sagni\Downloads\Disaster Prediction\disaster_prediction_heatmap.png"

# Load HDF5 file
with h5py.File(h5_path, 'r') as f:
    before = f['before_disaster'][:]  # shape: (bands, height, width)
    after = f['after_disaster'][:]

# Compute band-wise absolute difference
diff = np.abs(after - before)  # shape: (bands, H, W)

# Flatten spatial dimensions
bands, H, W = diff.shape
X = diff.reshape(bands, -1).T  # shape: (H*W, bands)

# Create dummy labels based on threshold (mean + std)
threshold = diff.mean() + diff.std()
labels = (X.mean(axis=1) > threshold).astype(int)  # 1 = disaster-affected, 0 = not

# Sample training data
X_train, X_test, y_train, y_test = train_test_split(X, labels, train_size=0.1, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save the model
joblib.dump(model, model_save_path)
print(f"✅ Model saved at: {model_save_path}")

# Predict full image
full_pred = model.predict(X)  # shape: (H*W,)
pred_map = full_pred.reshape(H, W)

# Visualize prediction heatmap
plt.figure(figsize=(10, 8))
plt.imshow(pred_map, cmap='Reds')
plt.title("Predicted Disaster-Affected Areas")
plt.axis('off')
plt.colorbar(label='Disaster Probability (0=No, 1=Yes)')
plt.savefig(prediction_map_path, bbox_inches='tight')
plt.close()

print(f"🗺️ Prediction heatmap saved at: {prediction_map_path}")


Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00  27525008
           1       1.00      1.00      1.00    251226

    accuracy                           1.00  27776234
   macro avg       1.00      1.00      1.00  27776234
weighted avg       1.00      1.00      1.00  27776234

✅ Model saved at: C:\Users\sagni\Downloads\Disaster Prediction\disaster_predictor.pkl
🗺️ Prediction heatmap saved at: C:\Users\sagni\Downloads\Disaster Prediction\disaster_prediction_heatmap.png
